In [1]:
from nltk.stem.porter import PorterStemmer
import string
import xml.dom.minidom 
import re
import math
import scipy
import csv
import pandas as pd
import numpy as np
import pickle
from gensim import models
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import datapath
import sklearn
from sklearn import svm
from sklearn.metrics import classification_report
from time import *

### Rank搜索

这一部分是读取index和xml，输入query（书名、作者名或描述），返回文件名。

In [55]:
class Search():
    
    def __init__(self, index_path,title_index_path,writer_index_path ,csv_path):
        #读index
        #Rank用list
        self.word_list = []
        self.Document_f = []
        self.word_document_number = []
        self.word_tf = []
        self.word_docu_position = []
        self.stop_words = []
        #书名用list
        self.title_list = []
        self.title_document_number = []
        self.title_docu_position = []
        #作者名用list
        self.writer_list = []
        self.writer_document_number = []
        self.writer_docu_position = []
        #读三个index
        self.index_path = index_path
        self.ReadIndex(index_path)
        self.ReadTIndex(title_index_path)
        self.ReadWIndex(writer_index_path)
        self.ReadCSV(csv_path)
        #for LSVM
        self.wordDic = {}
        self.ReadLSVM_dic()
        
            
    #读index函数
    def ReadIndex(self, index_path):
        word_list = []
        Document_f = []
        word_document_number = []
        word_tf = []
        dn = []
        tf = []

        with open(index_path,'r') as f :
            lines = f.readlines()
            for line in lines:
                if '\t' in line:
                    #re按照符号切分
                    DN_and_TF = re.split('([0-9]+)',line)
                    #filter只保留数字
                    DN_and_TF = list(filter(str.isdigit, DN_and_TF))
                    dn.append(DN_and_TF[0])
                    tf.append(len(DN_and_TF)-1)
                else:
                    word_document_number.append(dn)
                    word_tf.append(tf)
                    dn = []
                    tf = []
                    W_and_DF = re.split(r'[:()]',line)
                    W_and_DF = list(filter(str.isalnum, W_and_DF))
                    word_list.append(W_and_DF[0])
                    Document_f.append(W_and_DF[1])
        f.close()
        
        stop_words = []
        with open('englishST.txt','r') as f:
            lines = f.readlines()
            for line in lines:
                stop_words.append(line.strip('\n'))
        f.close()
        self.stop_words = stop_words
        
        word_document_number.pop(0)
        word_tf.pop(0)
        
        self.word_list = word_list
        self.Document_f = Document_f
        self.word_document_number = word_document_number
        self.word_tf = word_tf
        
    #读书名和作者的index，结构相同    
    def ReadTIndex(self, title_index_path):
        word_list = []
        word_document_number = []
        word_docu_position = []
        dp = []
        dn = []

        with open(title_index_path,'r') as f :
            lines = f.readlines()
            for line in lines:
                if '\t' in line:
                    #re按照符号切分
                    DN_and_TF = re.split('([0-9]+)',line)
                    #filter只保留数字
                    DN_and_TF = list(filter(str.isdigit, DN_and_TF))
                    dn.append(DN_and_TF[0])
                    dp.append(DN_and_TF[1:])
                else:
                    word_docu_position.append(dp)
                    word_document_number.append(dn)
                    dp = []
                    dn = []
                    W_and_DF = re.split(r'[:()]',line)
                    W_and_DF = list(filter(str.isalnum, W_and_DF))
                    word_list.append(W_and_DF[0])
        f.close()
        word_docu_position.pop(0)
        word_document_number.pop(0)
        
        self.title_list = word_list
        self.title_document_number = word_document_number
        self.title_docu_position = word_docu_position
    
    def ReadWIndex(self, writer_index_path):
        word_list = []
        word_document_number = []
        word_docu_position = []
        dp = []
        dn = []

        with open(writer_index_path,'r') as f :
            lines = f.readlines()
            for line in lines:
                if '\t' in line:
                    #re按照符号切分
                    DN_and_TF = re.split('([0-9]+)',line)
                    #filter只保留数字
                    DN_and_TF = list(filter(str.isdigit, DN_and_TF))
                    dn.append(DN_and_TF[0])
                    dp.append(DN_and_TF[1:])
                else:
                    word_docu_position.append(dp)
                    word_document_number.append(dn)
                    dp = []
                    dn = []
                    W_and_DF = re.split(r'[:()]',line)
                    W_and_DF = list(filter(str.isalnum, W_and_DF))
                    word_list.append(W_and_DF[0])
        f.close()
        word_docu_position.pop(0)
        word_document_number.pop(0)
        
        self.writer_list = word_list
        self.writer_document_number = word_document_number
        self.writer_docu_position = word_docu_position
    
    #读XML获取文件数量，整合代码可以直接使用self里的document进行文件提取
    def ReadCSV(self, csv_path):
        #这里只是为了获得有多少个文件
        with open(csv_path, 'r') as f:
            self.document_nub = len(f.readlines())
        f.close()
    
    
    '''
    搜索框启动器
    '''
    #搜索框启动器
    def Start(self):
        print('\nPlease enter the query:')
        self.query = input()
        #将query去符号并分割
        begin_time = time()
        falg = self.judgeQueries(self.query)
        queryL = self.excuteQueries(self.query)
        if flag == 'B':
            documents = self.BW_PhraseSearch(queryL,self.title_list,self.title_document_number,self.title_docu_position)
        elif flag == 'W':
            documents = self.BW_PhraseSearch(queryL,self.writer_list,self.writer_document_number,self.writer_docu_position)
        else:
            documents = self.RSearch(queryL)
        end_time = time()
        print("Search Type:",flag)
        print("搜索结果是：", documents[:10])
        print('Time is:', end_time-begin_time)
    
    '''
    LinearSVM判定query是作者名、书名或混合
    在class初始化环节完成模型的训练and save model，搜索框出现直接使用模型
    '''
    def LinearSVM(self, data_path):
        books_list = []
        writers_list = []
        des_list = []
        doc_text = []
        lable_list = []
        LSVM_word_list = []
        punct = re.compile(f'[{string.punctuation}]')
        data = pd.read_csv(data_path)
    
        books_list = list(data['title'])
        books_list = [i for i in books_list if i != ' ']
        for lines in books_list:
            text =  punct.sub('',str(lines)).split()
            for word in text:
                if len(word)!=0:
                    LSVM_word_list.append(word)
                else:
                    continue
            lines = text
            doc_text.append(lines)
            lable_list.append("B")

        writers_list = list(data['author'])
        writers_list = [i for i in writers_list if i != ' ']
        for lines in writers_list:
            text =  punct.sub('',str(lines)).split()
            for word in text:
                if len(word)!=0:
                    LSVM_word_list.append(word)
                else:
                    continue
            lines = text
            doc_text.append(lines)
            lable_list.append("W")

        des_list = list(data['description'])
        des_list = [i for i in des_list if i != ' ']
        for lines in des_list:
            text =  punct.sub('',str(lines)).split()
            for word in text:
                if len(word)!=0:
                    LSVM_word_list.append(word)
                else:
                    continue
            lines = text
            doc_text.append(lines)
            lable_list.append("D")


        LSVM_word_list = set(LSVM_word_list)
        wordDic = {}
        for word_id, wordc in enumerate(LSVM_word_list):
            wordDic[wordc] = word_id

        with open("LSVM_wordDic","w") as fi:
            for word in wordDic:
                fi.write(str(word)+"\n")
        fi.close()
            
        matrix_size = (len(doc_text),len(wordDic)+1)
        indeX = len(wordDic)
        X = scipy.sparse.dok_matrix(matrix_size)
        for doc_id, doc in enumerate(doc_text):
            for w in doc:
                X[doc_id,wordDic.get(w,indeX)] += 1

        train_X, test_X, y_train, y_test = sklearn.model_selection.train_test_split(X, lable_list, test_size=0.2, shuffle = True, stratify=lable_list)
        model = svm.LinearSVC(C = 0.1, loss = 'squared_hinge',multi_class='crammer_singer',penalty= 'l2')
        model.fit(train_X, y_train)
        file = open("model.pickle","wb")
        pickle.dump(model,file)
        file.close()
        prediction_t = model.predict(test_X)
        print("\nreport:")
        print(classification_report(y_test,prediction_t,target_names=['B','W','M']))
        
    '''
    下面是精彩的search组件
    '''
    #query切片
    def excuteQueries(self, query):
        queryWords = re.split(" ", query)
        query_word_list = []
        
        for word in queryWords:
            word = word.lower()
            for w in word:
                if w in string.punctuation:
                    word = word.replace(w,"")
            query_word_list.append(word)
            
        query_word_list = [i for i in query_word_list if i != '']        
        return query_word_list
    
    def ReadLSVM_dic(self):
        wordlist = []
        with open("LSVM_wordDic",'r') as fi:
            lines = fi.readlines()
            for line in lines:
                wordlist.append(str(line).strip())
        fi.close()
        wordDic = {}
        for word_id, wordc in enumerate(wordlist):
            wordDic[wordc] = word_id
        self.wordDic = wordDic
    
    def judgeQueries(self, query):
        punct = re.compile(f'[{string.punctuation}]')
        text =  punct.sub('',str(query)).split()
        wordDic = self.wordDic

        matrix_size = (1,len(wordDic)+1)
        X = scipy.sparse.dok_matrix(matrix_size)
        for w in text:
            X[0,wordDic.get(w)] += 1

        file = open("model.pickle", "rb")
        model = pickle.load(file)
        file.close()
        flag = model.predict(X)
        
        return flag
    
    '''
    混合Rank搜索_TFIDF
    '''
    def get_word_rank_score(self, wordR):
        #score = (1+lg(tf))*lg(dn/df)
        word_list = self.word_list
        worddf = self.Document_f
        dn = int(self.document_nub)

        scoreR = []
        log = math.log
        Numb = int(word_list.index(wordR))
        
        s_word_document_number = self.word_document_number[Numb]
        Df = float(worddf[Numb])
        Tf = self.word_tf[Numb]
        for i in range(0, len(s_word_document_number)):   
            Score = (1+log(int(Tf[i]),10))*log((dn/Df),10)
            Score = float(format(Score, '.4f'))
            scoreR.append(Score)
        return s_word_document_number,scoreR
    
    
    def RSearch(self, queryL):
        Wdocument_number = []
        WScore = []
        Dic_words_score = {}
        Rdocument = []
        Rscore = []
        query_word_list = []
        BDocuments_List = []
        porter_stemmer = PorterStemmer()
        stop_words = self.stop_words
        word_list = self.word_list
        Word_Documentno = self.word_document_number
        
        for word in queryL:
            if word not in stop_words:
                word = porter_stemmer.stem(word)
                query_word_list.append(word)
        queryL = query_word_list
  
        BDocuments_List = Word_Documentno[word_list.index(queryL[0])]
        for words in range(1, len(queryL)):
            tempL = Word_Documentno[word_list.index(queryL[words])]
            BDocuments_List = list(set(BDocuments_List).intersection(set(tempL)))
        if len(BDocuments_List)==1:
            Rdocument = BDocuments_List
        else:
            for word in queryL:
                Wdocument_number, WScore = self.get_word_rank_score(word)
                a = len(Wdocument_number)
                for y in range(0,a):
                    if Wdocument_number[y] not in Dic_words_score:
                        Dic_words_score[Wdocument_number[y]] = float(WScore[y])
                    else:
                        Dic_words_score[Wdocument_number[y]] = float(Dic_words_score[Wdocument_number[y]])+float(WScore[y])
    
            Dic_words_score = sorted(Dic_words_score.items(), key=lambda d: d[1], reverse=True)
            for term in Dic_words_score:
                Rdocument.append(term[0])
        
        return Rdocument
    
    
    '''
    书名作者名的phrase
    '''
    def BW_PhraseSearch(self, queryL,word_list,Word_Documentno,word_position):
        documentS = []
        CHA = {}
    
        #以第一个单词的documentL为基准点，取交集
        BDocuments_List = Word_Documentno[word_list.index(queryL[0])]
        for words in range(1, len(queryL)):
            tempL = Word_Documentno[word_list.index(queryL[words])]
            BDocuments_List = list(set(BDocuments_List).intersection(set(tempL)))
    
        #如果没有交集，取并集
        if len(BDocuments_List)==0:
            BDocuments_List = Word_Documentno[word_list.index(queryL[0])]
            for words in range(1, len(queryL)):
                tempL = Word_Documentno[word_list.index(queryL[words])]
                BDocuments_List = list(set(BDocuments_List).union(set(tempL)))
            documentS = BDocuments_List
        #如果有唯一交集，直接输出
        elif len(BDocuments_List)==1:
            documentS = BDocuments_List
        #如果有交集且不唯一，计算区间，优先近的文件
        else:
            print(BDocuments_List)
            for i in range(0,len(queryL)-1):
                posiL1 = word_position[word_list.index(queryL[i])]
                posiL2 = word_position[word_list.index(queryL[i+1])]
                donuL1 = Word_Documentno[word_list.index(queryL[i])]
                donuL2 = Word_Documentno[word_list.index(queryL[i+1])]
                for docu in BDocuments_List:
                    l1 = posiL1[donuL1.index(docu)]
                    l2 = posiL2[donuL2.index(docu)]
                    cha = []
                    for x in l1:
                        for y in l2:
                            z = abs(int(x)-int(y))
                            cha.append(z)
                    if 1 in cha:
                        documentS.append(docu)
                    else:
                        if str(docu) not in CHA:
                            CHA[str(docu)] = sum(cha)
                        else:
                            CHA[str(docu)] = CHA[str(docu)] + sum(cha)

        CHA = sorted(CHA.items(), key=lambda d: d[1], reverse=True)
        for term in CHA:
            documentS.append(term[0])
        documentS = list(set(documentS))

        return documentS

In [56]:
if __name__ == '__main__':
    index_path = "content.txt"
    csv_path = "GoodReads_processed5.csv"
    title_index_path = "title.txt"
    writer_index_path = "author.txt"
    Se = Search(index_path,title_index_path,writer_index_path ,csv_path)
    Se.Start()


Please enter the query:
Between Two Fires: American Indians in the Civil War
Search Type: ['B']
搜索结果是： ['1']
Time is: 0.028199434280395508


In [57]:
#LSVM training. Save model and wordList for query judgement.
if __name__ == '__main__':
    #CW1的index
    index_path = "content.txt"
    csv_path = "GoodReads_processed5.csv"
    title_index_path = "title.txt"
    writer_index_path = "author.txt"
    #输入CW1xml的headline和文件内容可以查询到得分前十的文件id.
    Se = Search(index_path,title_index_path,writer_index_path ,csv_path)
    Se.LinearSVM(csv_path)


report:
              precision    recall  f1-score   support

           B       0.97      0.90      0.93     19267
           W       1.00      0.98      0.99     19268
           M       0.91      0.99      0.95     19268

    accuracy                           0.96     57803
   macro avg       0.96      0.96      0.96     57803
weighted avg       0.96      0.96      0.96     57803



### 条件搜索

输入条件单词，搜索结果取两个单词的和（Boolean）

界面可以做成方框打钩（或者做的有创意点，摇一摇什么的，随机生成几个词条，然后摇一摇再随机生成之类花里胡哨的玩意），一共有30个种类，存放在range_list中。

注意，这里输入的query为：A,B,C   

的形式。前端可以将界面中选择的词条按这个形式输入，或者更改exQueries，最终queryL的形式是list：['A','B','C']

In [ ]:
class RangeSearch():
    def __init__(self,index_path):
        self.range_list = []
        self.documents_list = []
        self.ReadIndex(index_path)
    
    def Start(self):
        print('\nPlease enter the query:')
        self.query = input()
        begin_time = time()
        queryL = self.exQueries(self.query)
        print(queryL)
        documents = self.BooleanAndSearch(queryL)
        end_time = time()
        print("搜索结果是：", documents)
        print('Time is:', end_time-begin_time)
    
    
    def ReadIndex(self, index_path):
        range_list = []
        documents_list = []
        
        with open(index_path,'r') as file:
            lines = file.readlines()
            for line in lines:
                ss = re.split(r':',line)
                temp = re.split('([0-9]+)',ss[1])
                temp = list(filter(str.isdigit, temp))
                ss[1] = list(temp)
                range_list.append(ss[0])
                documents_list.append(ss[1])
        file.close()
        self.range_list = range_list
        self.documents_list = documents_list
        
    def exQueries(self, query):
        temp = re.split(r',',query)
        queryL = list(temp)
        return queryL
    
    def BooleanAndSearch(self,queryL):
        range_list = self.range_list
        documents_list = self.documents_list
        docuS = documents_list[range_list.index(queryL[0])]
        for i in range(1,len(queryL)):
            tempL = documents_list[range_list.index(queryL[i])]
            #这里取得是交集
            docuS = list(set(docuS).intersection(set(tempL)))
        
        #如果选了三个以上的标签会出现没那种书的情况，所以每两种取交集再取并集
        if len(docuS)==0:
            a = 2
            while (a < len(queryL)):
                tempL1 = documents_list[range_list.index(queryL[a])]
                tempL2 = documents_list[range_list.index(queryL[a-1])]
                tempL3 = list(set(tempL1).intersection(set(tempL2)))
                a += 1
                docuS = list(set(tempL3).union(set(docuS)))
        
        return docuS

In [ ]:
if __name__ == '__main__':
    index_path = "genre.txt"
    Se = RangeSearch(index_path)
    Se.Start()

Please enter the query:

Nonfiction,Classics,Sequential Art,Childrens
['Nonfiction', 'Classics', 'Sequential Art', 'Childrens']

搜索结果是： ['79118', '50049', '80377', '32338', '86708', '41465', '46247', '32134', '61221', '49387', '94992', '4993', '32177', '50968', '50985', '66414', '93451', '28301', '17690', '58802', '87047', '4012', '17695', '3187', '20915', '75287', '95614', '74462', '32164', '81225', '81786', '39320', '21134', '32176', '17613', '6360', '32324', '52025', '28006', '20913', '17676', '4750']

Time is: 0.003989696502685547

In [37]:
#ReadIndex 的BUG
word_list = []
Document_f = []
word_document_number = []
word_tf = []
dn = []
tf = []

with open("author.txt",'r') as f :
    lines = f.readlines()
    for line in lines:
        if '\t' not in line:
            W_and_DF = re.split(r'[:()]',line)
            W_and_DF = list(filter(str.isalnum, W_and_DF))
            word_list.append(W_and_DF[0])
            Document_f.append(W_and_DF[1])
            word_document_number.append(dn)
            word_tf.append(tf)
            dn = []
            tf = []
        else:
            #re按照符号切分
            DN_and_TF = re.split('([0-9]+)',line)
            #filter只保留数字
            DN_and_TF = list(filter(str.isdigit, DN_and_TF))
            dn.append(DN_and_TF[0])
            tf.append(len(DN_and_TF)-1)
f.close()

word_document_number.pop(0)
word_tf.pop(0)

#有个bug在这，最后一行没有读，所以后两个表比前两个表少一个值。。。我一直看不出来ORZ
print(word_list[-1])
print(Document_f[-1])
print(word_document_number[-1])
print(word_tf[-1])

matsumura
1
['96330']
[1]
